In [694]:
# warnings....ignore
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


pd.options.mode.chained_assignment = None

In [695]:
df = pd.read_csv("../statics/csv/cc_prom.csv")

### Clean data

In [696]:
# Select columns for easy following
cols = df.columns.to_list()
colums_drop = []
for col in cols:
    num_row = df.drop_duplicates(subset=col).__len__()
    if num_row == 1:
        colums_drop.append(col)
print(colums_drop)

['__name__', 'app', 'controller_class', 'controller_namespace', 'host', 'ingress', 'job', 'kubernetes_namespace', 'namespace', 'ns', 'path', 'pod_template_hash', 'service']


In [697]:
df = df.drop(columns=colums_drop) # Only select column selected
df.head()

,Time,controller_pod,instance,kubernetes_node,kubernetes_pod_name,le,method,status,Value #A
0,1707116670000,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
1,1707116700000,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
2,1707116730000,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
3,1707116760000,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
4,1707116790000,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13


In [698]:
# Change columns name
# rename columns
cols = {
    "Time": "timestamp",
    "Value #A": "value"
}
df = df.rename(columns=cols)
df['timestamp'] = pd.to_datetime(df["timestamp"], unit="ms")
df.head()

,timestamp,controller_pod,instance,kubernetes_node,kubernetes_pod_name,le,method,status,value
0,2024-02-05 07:04:30,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
1,2024-02-05 07:05:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
2,2024-02-05 07:05:30,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
3,2024-02-05 07:06:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13
4,2024-02-05 07:06:30,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,DELETE,308,13


In [699]:
sample_props = ["controller_pod", "instance", "kubernetes_node", "kubernetes_pod_name", "le", "method", "status"]

In [700]:
# Exact columns
sample_group = df.groupby(sample_props).last().reset_index()
print(f"Have {len(sample_group)} samples")
sample_group.head()


Have 1548 samples


,controller_pod,instance,kubernetes_node,kubernetes_pod_name,le,method,status,timestamp,value
0,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,2024-02-05 07:44:30,1
1,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,405,2024-02-05 07:44:30,0
2,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,GET,200,2024-02-05 07:44:30,75810908
3,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,GET,206,2024-02-05 07:44:30,8158
4,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,GET,304,2024-02-05 07:44:30,8314636


In [701]:
# Create df group data in 5 min
df_grouped = df.groupby([pd.Grouper(freq="5min", key="timestamp")] + sample_props).first().reset_index()
print("Total row: ", len(df_grouped))
df_grouped.head()

Total row:  13932


,timestamp,controller_pod,instance,kubernetes_node,kubernetes_pod_name,le,method,status,value
0,2024-02-05 07:00:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1
1,2024-02-05 07:00:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,405,0
2,2024-02-05 07:00:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,GET,200,75790945
3,2024-02-05 07:00:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,GET,206,8155
4,2024-02-05 07:00:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,GET,304,8314355


In [702]:
# Implement rate calculation for each group
list_df_rated = []


def rate(group: pd.Series):
    sample = df_grouped[(df_grouped["controller_pod"] == group['controller_pod'])
                        & (df_grouped["instance"] == group['instance'])
                        & (df_grouped["kubernetes_node"] == group['kubernetes_node'])
                        & (df_grouped["kubernetes_pod_name"] == group['kubernetes_pod_name'])
                        & (df_grouped["le"] == group['le'])
                        & (df_grouped["method"] == group['method'])
                        & (df_grouped["status"] == group['status'])].reset_index(drop=True)
    sample['rate'] = sample['value'].diff(
    ) / sample['timestamp'].diff().dt.total_seconds()
    rate_index_2 = sample['rate'][sample.index == 1]
    sample.at[0, 'rate'] = rate_index_2
    sample.fillna(0, inplace=True)
    list_df_rated.append(sample)


for idx, group in sample_group.iterrows():
    rate(group)

df_result = pd.concat(list_df_rated)
df_result.head(5)

,timestamp,controller_pod,instance,kubernetes_node,kubernetes_pod_name,le,method,status,value,rate
0,2024-02-05 07:00:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0
1,2024-02-05 07:05:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0
2,2024-02-05 07:10:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0
3,2024-02-05 07:15:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0
4,2024-02-05 07:20:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0


In [706]:
df_sum_rate = df_result
df_sum_rate['sum'] = df_sum_rate.groupby(['timestamp', 'controller_pod', 'instance', 'kubernetes_node', 'kubernetes_pod_name', 'le', 'method', 'status'])['rate'].cumsum()
df_sum_rate

,timestamp,controller_pod,instance,kubernetes_node,kubernetes_pod_name,le,method,status,value,rate,sum
0,2024-02-05 07:00:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0,0.0
1,2024-02-05 07:05:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0,0.0
2,2024-02-05 07:10:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0,0.0
3,2024-02-05 07:15:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0,0.0
4,2024-02-05 07:20:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.96:10254,br-ingress2v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,0.005,DELETE,308,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4,2024-02-05 07:20:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.95:10254,br-ingress1v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,PUT,405,304,0.0,0.0
5,2024-02-05 07:25:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.95:10254,br-ingress1v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,PUT,405,304,0.0,0.0
6,2024-02-05 07:30:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.95:10254,br-ingress1v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,PUT,405,304,0.0,0.0
7,2024-02-05 07:35:00,ingress-nginx-controller-browser-prod-6667d78f...,172.16.16.95:10254,br-ingress1v.itim.vn,ingress-nginx-controller-browser-prod-6667d78f...,inf,PUT,405,304,0.0,0.0


In [712]:
df_sum_rate.query("le == 0.01 and timestamp == '2024-02-05 07:20:00'")['rate'].sum()

26.60333333333333